# 단방향 암호화

## 복화화가 불가능한 암호화 기법

In [1]:
import hashlib

In [7]:
str1 = "홍길동"
str2 = "hongkildong"

result = hashlib.sha256(str1.encode())
print("'",str1, "' 의 해시값(sha256) : ", result.hexdigest())

result = hashlib.sha256(str2.encode())
print("'",str2, "' 의 해시값(sha256) : ", result.hexdigest())

' 홍길동 ' 의 해시값(sha256) :  9c6b9b1b1627f3120e0730c6d2cfa71040fd03747bde2755e8b5e4dbf2bee262
' hongkildong ' 의 해시값(sha256) :  2ea65ba4c4d7fd61965ae2c8e111dfec04f001afc666a12878d3fa99e8cb0c2e


 # 양방향 암호화 대칭키 방식

## 대칭키 암호화의 가장 대표적인 방식은  AES암호화 알고리즘입니다. 그렇기 때문에 공개키에 비해 매우 빠르게 수행되는 장점이 있는 반면 키를 상대방에게 전달할때 해커들에게 가로채기 등을 당할 수 있어 키교환(키관리) 문제가 필연적으로 발생하게 됩니다.

In [19]:
import base64
import hashlib
from Crypto.Cipher import PKCS1_OAEP

## AES객체(키) 생성

In [23]:
import hashlib
from Crypto.Cipher import PKCS1_OAEP

class AESCipher(object):
    def __init__(self, key):
        self.key = hashlib.sha256(key.encode()).digest()
        print("AES KEY(Key문장 암호화):", self.key)
        
print("-"*100, "\n")
key = "aesKey"
msg = "원본 메세지 입니다."
print("AES KEY: ", key)
print("원본메세지: ", msg)

aes = AESCipher(key)

---------------------------------------------------------------------------------------------------- 

AES KEY:  aesKey
원본메세지:  원본 메세지 입니다.
AES KEY(Key문장 암호화): b'J\x8et\xae\x0fo\xed\xd7\xa4{\x93r\xc9\xb9\xefG%\xdc\xa0\x95\xeb\x87R\x89I\x14\x91\x97\x9b\xae\xbc\x12'


## msg를 AES 암호화 알고리즘 방식으로 암호화

In [35]:
import base64
import hashlib
from Crypto.Cipher import AES

BS = 16 # block size를 16바이트로 고정시켜놔야함 (AES 특징)
#입력 받은 데이터의 길이가 블럭사이즈의 배수가 아닐때 아래와 같이 패딩을 해주어야 한다.
#패딩 : 데이터의 길이가 블럭사이즈의 배수가 아닐때 마지막 블럭값을 추가해 블럭사이즈의 배수로 맞추어 주는 행위
pad = (lambda s : s+ (BS - len(s) % BS) * chr(BS-len(s) % BS).encode())
unpad = (lambda s: s[:-ord(s[len(s)-1:])])
print(pad)

class AESCipher(object):
    def __init__(self, key):
        self.key = hashlib.sha256(key.encode()).digest()
        print("AES KEY(key 문장 암호화) : ", self.key)
        
    def encrypt(self, message): # 암호화 함수
        message = message.encode()
        raw = pad(message)
        cipher = AES.new(self.key, AES.MODE_CBC, self.__iv().encode('utf8'))
        enc = cipher.encrypt(raw)
        return base64.b64encode(enc).decode('utf-8')
    
    def __iv(self):
        return chr(0) * 16
    
print("-"*100, "\n")
key = "aeskey"
msg = "원본 메시지 입니다."
print("AES KEY: ", key)
print("원본 메시지: ", msg)

aes = AESCipher(key)
encrypt = aes.encrypt(msg)
print("_"*100, "\n")
print("원본 메시지를 AES키로 암호화한 결과: ", encrypt)
print("_"*100, "\n")


<function <lambda> at 0x7efcf0400ee0>
---------------------------------------------------------------------------------------------------- 

AES KEY:  aeskey
원본 메시지:  원본 메시지 입니다.
AES KEY(key 문장 암호화) :  b'\xf6`$\x9fR\x1e\x81\x99\xc7\xe6\xa0\xd4\xcf\n\xa4\xdd\x12qz\xd9)\xff\xc7Z\x15\x83 \xc8\x86\xafy4'
____________________________________________________________________________________________________ 

원본 메시지를 AES키로 암호화한 결과:  QMrSBaxHMnJC7GFCUXzuRy+a7PQX+Y9oGl1Y/x46RBM=
____________________________________________________________________________________________________ 



## 암호화된 메세지를 다시 복호화

In [39]:
BS = 16
pad = (lambda s: s+ (BS - len(s) % BS) * chr(BS - len(s) % BS).encode())
unpad = (lambda s : s[:-ord(s[len(s)-1:])])

class AESCipher(object):
    def __init__(self, key):
        self.key = hashlib.sha256(key.encode()).digest()
        print("AES Key(Key 문장 암호화) : ", self.key)
        
    def encrypt(self, message):
        message = message.encode()
        raw = pad(message)
        cipher = AES.new(self.key, AES.MODE_CBC, self.__iv().encode('utf8'))
        enc = cipher.encrypt(raw)
        return base64.b64encode(enc).decode('utf-8')
    

    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        cipher = AES.new(self.key, AES.MODE_CBC, self.__iv().encode('utf8'))
        dec = cipher.decrypt(enc)
        return unpad(dec).decode('utf-8')
    
    def __iv(self):
        return chr(0) * 16
    
print("-"*100, "\n")
key = "aesKey"
msg = "원본 메시지 입니다."
print("AES KEY: ", key)
print("원본 메시지: ", msg)


aes = AESCipher(key) # 1. 대칭키 암복호화 처리를 위해 AESCipher클래스의 객체(인스턴스)를 생성(해시(256bit)가 적용된 키값을 얻어옴)
# print(aes)

encrypt = aes.encrypt(msg) # 2.입력한 메시지를 AES 대칭키 암호화 방식으로 암호화
print("_"*100, "\n")
print("메시지 원본을 aes키로 암호화한 결과: ", encrypt)
print("_"*100, "\n")


decrypt = aes.decrypt(encrypt) # 3.암호화된 메시지를 AES 대칭키 암호화 방식으로 복호화
print("암호화된 메시지를 복호화한 결과: ", decrypt) 
print("_"*100, "\n")


---------------------------------------------------------------------------------------------------- 

AES KEY:  aesKey
원본 메시지:  원본 메시지 입니다.
AES Key(Key 문장 암호화) :  b'J\x8et\xae\x0fo\xed\xd7\xa4{\x93r\xc9\xb9\xefG%\xdc\xa0\x95\xeb\x87R\x89I\x14\x91\x97\x9b\xae\xbc\x12'
____________________________________________________________________________________________________ 

메시지 원본을 aes키로 암호화한 결과:  goCOr7RH6es7/jnBdIa4b4Uh8ghSx8J2aXQHh/+LAIo=
____________________________________________________________________________________________________ 

암호화된 메시지를 복호화한 결과:  원본 메시지 입니다.
____________________________________________________________________________________________________ 



# 양방향 암호화 비대칭키 방식(RSA방식)

In [50]:
import base64
from Crypto.PublicKey import RSA

In [51]:
## 공개키 및 프라이빗 키 생성

In [52]:
def make_key():
    pr_key = RSA.generate(2048)
    print(pr_key)
    pu_key = pr_key.public_key()
    print(pu_key)
    
    pr_file = open('pr.key', 'wb')
    pr_file.write(pr_key.export_key('PEM'))
    pr_file.close()
    
    pu_file = open('pu.key', 'wb')
    pu_file.write(pu_key.export_key('PEM'))
    pu_file.close()
    
make_key()

Private RSA key at 0x7EFCF0346820
Public RSA key at 0x7EFCF03F6BB0


## 퍼블릭키로 메시지를 암호화

In [45]:
import base64
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

In [56]:
#key 생성 및 저장

def make_key():
    pr_key = RSA.generate(2048) #2048길이의 키 생성
    print(pr_key)
    pu_key = pr_key.public_key()
    print(pu_key)
    
    pr_file = open('pr_key', 'wb')
    pr_file.write(pr_key.export_key('PEM'))
    pr_file.close()
    
    pu_file = open('pu_key', 'wb')
    pu_file.write(pu_key.export_key('PEM'))
    pu_file.close()
    

#메시지 암호화
def encrypt_msg(msg, key):
    tool = PKCS1_OAEP.new(key)
    msg_enc = tool.encrypt(msg)
    
    return msg_enc

#메시지 복호화
def decrypt_msg(msg, key):
    tool = PKCS1_OAEP.new(key)
    msg_dec = tool.decrypt(msg)
    
    return msg_dec

#파일에서 키 가져오기
def get_key(path):
    fr = open(path, 'rb')
    key = RSA.importKey(fr.read())
    
    return key

def main():
    msg = 'hi name is aaaa'
    pu_key = get_key('pu.key')
    msg_enc = encrypt_msg(msg.encode(), pu_key)
    
    print("원문", msg)
    print("암호화 길이", len(msg_enc))
    print("암호화", msg_enc)
    
main()
    
    

원문 hi name is aaaa
암호화 길이 256
암호화 b"X5\x16HlqbW\xba\x87\xa3\x95E\xe6b\x1e\x8f=\xf2\x1a\xc9\\R\x9a\x07\x117\xd1rd\x92\x0f\xd6\xf1\xda\x93\xc4}v\xe7q\x1a\x19\xa4\x9c\x89@1\x11/\x03px\xe6\xdcC\xfc\x1f\x90\xb7V\rd\xe4\xb0[\x18h\x0c\x1e\xffn\x1a\xc8\x12f\xed\t\xe1\xca\xbbb\xa7Q9\xcc\x9f\xe3\xec\xd1[\xa5\x8f\xe4'\x1fp\xe71,]m\x93\xfc\x88d/\xa8\x15oQ\xa2\xf9\xe6\xcfl\xb1F\xbd\xb6\x06\x8e\xc5f\xce\xc5\x04\x8a\xe9\xbe\xae\x01\xe9\x04w\x19|\xbf\xed\x1d\x9e\xc3\x10\xc5\xad\xd4\x17\xd9\\\xaaV\xd6R\xa4\xd0\xad\xb8\xda\x01\xc3\xbb\x11\xc4\xa9\xee\x00\x9a1\xa7\xb1\x9dT1F\x97\xa0\xae\xf0\n0\x8e\x03\xaa\xb0\x0e\xde\x0c|\x03\x03\x1eM\xfa\x1d\x15\xf9nNF\xe4\x97\x84:T\xf1\xa7\xcf\x14\xf5\xc4\x80\xf2\x99\xfaFJ\x862E5\x1c\x88\xb6\x03\xd2\x13\x9f\xa7j\xb2\xb1\x92\xdd\xf4\x98Np\xed\xc6\x9d\x8f=\x00\xb8\x01\xdf6\x07\x80e\xf7\x85o[Uh"


## 퍼블릭 키로 암호화된 메시지를 프라이빗키로 복호화

In [58]:
import base64
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

In [61]:
def make_key():
    pr_key = RSA.generate(2048)
    print(pr_key)
    pu_key= pr_key.public_key()
    print(pu_key)
    
    pr_file = open('pr.key', 'wb')
    pr_file.write(pr_key.export_key('PEM'))
    pr_file.close()
    
    pu_file = open('pu.key', 'wb')
    pu_file.write(pu_key.export_key('PEM'))
    pu_file.close()
    
def encrypt_msg(msg, key):
    tool = PKCS1_OAEP.new(key)
    msg_enc = tool.encrypt(msg)
    
    return msg_enc

def decrypt_msg(msg, key):
    tool = PKCS1_OAEP.new(key)
    msg_dec = tool.decrypt(msg)
    
    return msg_dec

def get_key(path):
    fr = open(path, 'rb')
    key = RSA.importKey(fr.read())

    return key

def main():
    msg = 'this is crypto'
    pu_key = get_key('pu.key')
    msg_enc = encrypt_msg(msg.encode(), pu_key)
    
    print("원문", msg)
    print("암호화 길이", len(msg_enc))
    print("암호화", msg_enc)

    pr_key = get_key('pr.key')
    msg_dec = decrypt_msg(msg_enc, pr_key)
    
    print("복호화",msg_dec)
    
main()

원문 this is crypto
암호화 길이 256
암호화 b'\x8d\xee\xb4?pPh\x07\xba.\xdcL\xf3U\xe3[\xb4\xc1\xb2v\xddY\xecDH\xe6\xaa\xfb\xd0G\x07\xa3\xb0_\x9f,\x14\x02\xc63W\xae:O\x18]O\xa1\x11n\x02|\xdf\x95\x9e\xe6\x07\xcc"|;\xc5\xef\x05i.\xcc\x9cb7d\x89\xe1\x07\x04\x9b\xb6)\x93\xa1\xf0\x7fd\xe5\xc6\x08/\xfd\x97\xe3\x8b\x8e$Chl\xa5\xb254\x9a\xee\x7fI\x88{\xa9\xa7\x9a\x89\xc5\x12\xe7\t(_hP\xefd\xbc\xbf\xc2\x9f\x07\xf0/\xf4*0N\x15\x15\xb7\x111m\xc365\xf6v\xe8\xd1\xce\xe2l\xf7-b\x16\xcf/G\xc24\xda\x12\x15\xf7^\xa5\x01\x82o\xfe@\xe0|\xab\xd5\x8a[\x82\x85(\xc6\xad \xddI\x06\xd9\xfeE\xa9U\xe8^\t0W\x1dE\xad\xfcMI\x1e2n\xed\xc7\xe4\xad:t#g\xd8\x84\x9e\xc0\xc4\xba\x97\xefI\xd4);,2i\x94Wq\xac\x85\xf8A\xd1\xcb\nu\x84\x1b8\x0b\x88\xe5\xa5\xfc~\xf3\x00Y?K?0F\xd0\xa6\xdf\xa0'
복호화 b'this is crypto'
